In [ ]:
# !pip install ibm-watson-machine-learning==1.0.311 \
# python-dotenv==1.0.0 \
# requests==2.31.0 \
# urllib3==1.26.11 \
# rouge==1.0.1 \
# scikit-learn==1.2.2 \
# sentence-transformers==2.2.2 

# !pip install --upgrade "ibm-watson>=7.0.0"
# !pip install text-extensions-for-pandas


In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import text_extensions_for_pandas as tp
import pandas as pd
from dotenv import load_dotenv

In [ ]:
import os
os.environ['IBM_CLOUD_API_KEY'] = 'IBM_CLOUD_API_KEY'
os.environ['WATSONX_AI_ENDPOINT'] = 'https://us-south.ml.cloud.ibm.com'
os.environ['PROJECT_ID'] = '5a2ba826-ebd5-41fe-addc-1d8880d67ac7'

os.environ['WD_API_KEY']='WD_API_KEY'
os.environ['WD_ENDPOINT']='https://api.us-south.discovery.watson.cloud.ibm.com/instances/7978950c-ea5c-4ecb-9e74-1d4215917e6e'
os.environ['WD_PROJECT_ID']='WD_PROJECT_ID'


In [ ]:
discovery = None
WD_VERSION = '2023-03-31'
WD_PROJECT_ID = os.getenv("WD_PROJECT_ID", None)
WD_COLLECTION_ID = os.getenv("WD_COLLECTION_ID", "cc766481-368e-8358-0000-0181801be5b2")

In [ ]:
def init_wd():
    WD_API_KEY = os.getenv("WD_API_KEY", None)
    WD_ENDPOINT = os.getenv("WD_ENDPOINT", None)
    if WD_API_KEY is None or WD_ENDPOINT is None:
        print("Either api_key or api_url is None. Please make sure your credentials are correct.")
    
    authenticator = IAMAuthenticator(WD_API_KEY)
    global discovery
    discovery = DiscoveryV2(
        version=WD_VERSION,
        authenticator=authenticator
    )
    return discovery

In [ ]:
def query(payload: any):
    global discovery
    if(discovery is None):
        discovery = init_wd()
    discovery_results = discovery.query(
        project_id = payload['project_id'],
        count = payload['count'],
        natural_language_query = payload['natural_language_query'],
        filter = payload['filter'],
        sort = payload['sort'],
        collection_ids = payload['collection_ids'],
        return_ = payload['return']
    ).get_result()
    
    return discovery_results

In [ ]:
payload = {
    "project_id": "02afa05d-6eb2-47ad-9b99-52fde0e8e029",
    "count": 5,
    "natural_language_query": "Is it energy efficient ?",
    "filter": "product_number: 311996219,enriched_body.sentiment.label:positive",
    "sort": "-enriched_body.sentiment.score",
    "collection_ids": ["cc766481-368e-8358-0000-0181801be5b2"],
    "return": ["product_number", "title", "body", "score", "enriched_body.sentiment"]
}
discovery_results = query(payload)



In [ ]:
import json

if "results" in discovery_results and len(discovery_results['results']) > 0:
    json_formatted_str = json.dumps(discovery_results['results'], indent=2)
#     print(json_formatted_str)
    for result in discovery_results['results']:
        print(result['body'][0])
    